In [1]:
import os
import lpips
import cv2
import torch
import numpy as np
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim

In [3]:
# Get the list of image files in the folder
image_files = ["256x256.png", "512x512.png", "1000x1000.png", "2000x2000.png"]

# Loop over each image and convert to grayscale
for image_file in image_files:
    # Open the image
    img = Image.open(os.path.join('assets', image_file))

    # Convert the image to grayscale
    grayscale_img = img.convert("L")  # 'L' mode stands for grayscale

    # Save the grayscale image to the output folder
    grayscale_img.save(os.path.join('assets', f"grayscale_{image_file}"))

# Task 1

In [4]:
grayscaled_images = ["grayscale_256x256.png", "grayscale_512x512.png", "grayscale_1000x1000.png", "grayscale_2000x2000.png"]
for image_file in grayscaled_images:
    # Open the PNG image
    img = Image.open(f"assets/{image_file}")

    # Convert to JPEG format and save with compression
    img = img.convert("RGB")
    img.save(f"assets/{image_file}_compressed_q85.jpg", "JPEG", quality=85)  # 'quality' ranges from 1 (worst) to 95 (best)
    img.save(f"assets/{image_file}_compressed_q5.jpg", "JPEG", quality=5)  # 'quality' ranges from 1 (worst) to 95 (best)

# Task 2

In [5]:
grayscaled_images = ["grayscale_256x256.png", "grayscale_512x512.png", "grayscale_1000x1000.png", "grayscale_2000x2000.png"]

for image_file in grayscaled_images:
    # Open the PNG image
    img = Image.open(f"assets/{image_file}")
    img = img.convert("RGB")  # Convert to RGB format if necessary

    # Convert image to numpy array
    img_array = np.array(img)

    for stddev in [25, 85]:
        # Define Gaussian noise parameters
        mean = 0

        # Generate Gaussian noise
        gaussian_noise = np.random.normal(mean, stddev, img_array.shape)

        # Add the Gaussian noise to the image
        noisy_img_array = img_array + gaussian_noise

        # Ensure pixel values are within [0, 255] range
        noisy_img_array = np.clip(noisy_img_array, 0, 255).astype(np.uint8)

        # Convert back to a PIL image
        noisy_img = Image.fromarray(noisy_img_array)

        # Save the noisy image
        noisy_img.save(f"assets/gaussian{stddev}_{image_file}")


# Task 3

In [ ]:
# Function to compute MSE between two images
def mse(imageA, imageB):
    # Compute the mean squared error
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    
    return err

# Load images (grayscale images)
original = cv2.imread("assets/grayscale_256x256.png", cv2.IMREAD_GRAYSCALE)
original1 = cv2.imread("assets/grayscale_256x256.png", cv2.IMREAD_GRAYSCALE)
jpeg_compressed = cv2.imread("assets/grayscale_256x256.png_compressed_q85.jpg", cv2.IMREAD_GRAYSCALE)
gaussian_noise = cv2.imread("assets/gaussian85_grayscale_256x256.png", cv2.IMREAD_GRAYSCALE)

# Calculate MSE between the original and distorted images
mse_test = mse(original, original1)
mse_jpeg = mse(original, jpeg_compressed)
mse_noise = mse(original, gaussian_noise)

print(f"MSE for image: {mse_test}")
print(f"MSE for JPEG compressed image: {mse_jpeg}")
print(f"MSE for image with Gaussian noise: {mse_noise}")

In [ ]:
# Function to compute PSNR
def psnr(imageA, imageB):
    mse_value = mse(imageA, imageB)
    if mse_value == 0:
        return float('inf')  # PSNR is infinite if MSE is zero (identical images)
    max_pixel_value = 255.0
    return 10 * np.log10((max_pixel_value ** 2) / mse_value)

psnr_jpeg = psnr(original, jpeg_compressed)
psnr_noise = psnr(original, gaussian_noise)
print(f"PSNR for JPEG compressed image: {psnr_jpeg} dB")
print(f"PSNR for image with Gaussian noise: {psnr_noise} dB")

In [ ]:
ssim_jpeg = ssim(original, jpeg_compressed)
ssim_noise = ssim(original, gaussian_noise)

print(f"SSIM for JPEG compressed image: {ssim_jpeg}")
print(f"SSIM for image with Gaussian noise: {ssim_noise}")

In [ ]:
original = cv2.imread("assets/grayscale_256x256.png", cv2.IMREAD_GRAYSCALE)
original1 = cv2.imread("assets/grayscale_256x256.png", cv2.IMREAD_GRAYSCALE)
jpeg_compressed = cv2.imread("assets/grayscale_256x256.png_compressed_q85.jpg", cv2.IMREAD_GRAYSCALE)
gaussian_noise = cv2.imread("assets/gaussian85_grayscale_256x256.png", cv2.IMREAD_GRAYSCALE)

# Convert grayscale to RGB by repeating the channel and normalize
original_rgb = cv2.cvtColor(original, cv2.COLOR_GRAY2RGB) / 255.0
jpeg_compressed_rgb = cv2.cvtColor(jpeg_compressed, cv2.COLOR_GRAY2RGB) / 255.0
gaussian_noise_rgb = cv2.cvtColor(gaussian_noise, cv2.COLOR_GRAY2RGB) / 255.0

# Convert NumPy arrays to PyTorch tensors
original_tensor = torch.from_numpy(original_rgb).permute(2, 0, 1).float()  # Shape: (C, H, W)
jpeg_compressed_tensor = torch.from_numpy(jpeg_compressed_rgb).permute(2, 0, 1).float()
gaussian_noise_tensor = torch.from_numpy(gaussian_noise_rgb).permute(2, 0, 1).float()

lpips_model = lpips.LPIPS(net='vgg')

# Calculate LPIPS
lpips_jpeg = lpips_model(original_tensor.unsqueeze(0), jpeg_compressed_tensor.unsqueeze(0)).item()  # Add batch dimension
lpips_noise = lpips_model(original_tensor.unsqueeze(0), gaussian_noise_tensor.unsqueeze(0)).item()

print(f"LPIPS for JPEG compressed image: {lpips_jpeg}")
print(f"LPIPS for image with Gaussian noise: {lpips_noise}")